<a href="https://colab.research.google.com/github/kishormishra3/DeepLearn/blob/master/IIIT_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import pandas as pd
import numpy as np
import re
from sklearn import svm
from sklearn.model_selection import train_test_split

def cleantext(input_w, pattern):
    r = re.findall(pattern = pattern, string  = input_w)
    for i in r:
        input_w = re.sub(pattern = i, repl ='', string = input_w)
    return input_w


df = pd.read_csv('/gdrive/My Drive/train.csv')
df1_orig = pd.read_csv('/gdrive/My Drive/test.csv')

df1 = df1_orig.sample(n = 1000, random_state = 42)

vect = np.vectorize(cleantext)
fake = df.loc[df['label'] == 1]
#print("fake = ", fake.shape)
#combining both train and test dataset to perfrom preprocessing 
non_fake = df.loc[df['label'] == 0].sample(n = 2000, random_state = 42)
#print("non-fake = " , non_fake.shape)
normalized_df = pd.concat([fake, non_fake])
#normalized_df.head()
#print(normalized_df.shape)
combine  = normalized_df
combine['cleanedText'] = vect(combine['tweet'],'@[\w]*')
combine.head()
#remove punctuation
combine['cleanedText'] = combine['cleanedText'].str.replace("[^a-zA-Z]"," ")
combine.head()

#remove words less than word length <3
combine['cleanedText'] = combine['cleanedText'].str.replace(r'\b(\w{1,2})\b','')
combine.head()
tokenized_tweets = combine['cleanedText'].apply(lambda x:x.split())

#this gives me tokens of each tweet
#now apply stemming on these tokens to get the root word of every token

from nltk.stem import PorterStemmer

#call object of that class
stemmer = PorterStemmer()

tokenized_tweets = tokenized_tweets.apply(lambda x: [stemmer.stem(i) for i in x])

tokenized_tweets.head()


#now join the tokens of each tweet to form a preprocessed tweet

for i in range(len(tokenized_tweets)):
    tokenized_tweets[i] = ' '.join(tokenized_tweets[i])
combine['cleanedText'] = tokenized_tweets

label = df['label']


combine.head()
tweet  =  list((combine['cleanedText']))
file = '/gdrive/My Drive/glove.twitter.27B.200d.txt'
d= {}
glove = open(file,"r+")
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    d[word] = vector
glove.close()
vec = np.zeros([label.shape[0],200], dtype = 'float32') 
c=0
for i in tweet:
    for j in i:
        try:
            j=str(j)
            k=d[j]
            vec[c]=(vec[c]+np.array(k))
        except:
            continue
    c=c+1
X_train,X_test,y_train,y_test= train_test_split(vec,label,test_size=0.1,random_state=42)


In [0]:
clf=svm.SVC()
clf.fit(X_train,y_train)
prediction=clf.predict(X_test)
pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
print ('SVM accuracy=',(acc)/len(pred_labels)*100,'%')

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM accuracy= 94.71379418204566 %


In [0]:
from sklearn.neighbors import KNeighborsClassifier
clf =KNeighborsClassifier()         
clf.fit(X_train,y_train)
prediction=clf.predict(X_test)
pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
print ('KNN accuracy=',(acc)/len(pred_labels)*100,'%')

KNN accuracy= 93.05598999061621 %


In [0]:
from sklearn.linear_model import LogisticRegression
clf =LogisticRegression(random_state=42, solver ='saga')         
clf.fit(X_train,y_train)
prediction=clf.predict(X_test)
pred_labels=prediction==y_test
acc=0.0
for i in pred_labels:
    if i==True:
        acc+=1
print ('Logistic accuracy=',(acc)/len(pred_labels)*100,'%')

Logistic accuracy= 93.27494526118237 %


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
